In [1]:
ini_date = '04/01/23 12:00:00'
end_date = '06/01/23 13:00:00'

tt_date = '05/01/23 14:16:00'

time_step=15


target_folder="./time-data/evaluation/"

binary_sensors=["rule-cooking"]

#binary_sensors=["cubiertos", "vasos", "fridge", "lavadora", "microwave", "ollas", "trash", "tap"]

tags=["1","2"]

colors={}
colors["1"]="red"
colors["2"]="blue"
colors["3"]="orange"
colors["4"]="green"
colors["5"]="pink"
colors["6"]="brown"



In [2]:
def bb_intersection_over_union(boxA, boxB):
    if(not(boxB[0]<boxA[0] and boxB[1]>boxA[1])):
        if(boxB[0]<boxA[0]):
            return bb_intersection_over_union(boxB,boxA)
        if(boxB[1]<boxA[1]):
            return bb_intersection_over_union(boxB,boxA)
        if(boxB[0] < boxA[0] or boxB[0] > boxA[2]):
            return 0
        if(boxB[1] < boxA[1] or boxB[1] > boxA[3]):
            return 0
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    
    print(boxA,boxB," iou:",[xA,yA,xB,yB])

    # compute the area of intersection rectangle
    interArea = (xB - xA) * (yB - yA)

    
    if (xA>xB):
        return 0
    if (yA>yB):
        return 0    
    
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    if (iou<0):
        return 0
    # return the intersection over union value
    return interArea
bb_intersection_over_union([10, 20, 30, 50],[5, 10, 11, 21] )

[5, 10, 11, 21] [10, 20, 30, 50]  iou: [10, 20, 11, 21]


1

In [3]:
 def get_iou(bb1, bb2):
    assert bb1[0] <= bb1[2]
    assert bb1[1] <= bb1[3]
    assert bb2[0] <= bb2[2]
    assert bb2[1] <= bb2[3]

    # determine the coordinates of the intersection rectangle
    x_left = max(bb1[0], bb2[0])
    y_top = max(bb1[1], bb2[1])
    x_right = min(bb1[2], bb2[2])
    y_bottom = min(bb1[3], bb2[3])

    print(" iou:",[x_left,y_top,x_right,y_bottom])
    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    
    # compute the area of both AABBs
    bb1_area = (bb1[2] - bb1[0]) * (bb1[3] - bb1[1])
    bb2_area = (bb2[2] - bb2[0]) * (bb2[3] - bb2[1])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    #iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    print(" intersection_area/bb1_area:",intersection_area,bb1_area)
    if intersection_area==0 or bb1_area==0:
        return 1
    if(intersection_area==bb1_area):
        return 1
    iou = intersection_area / float(bb2_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou
get_iou([22, 25, 22, 26],[10, 20, 30, 50] )

 iou: [22, 25, 22, 26]
 intersection_area/bb1_area: 0 0


1

In [4]:
#get_iou([158, 544, 277, 403],[383, 371, 491, 451])

In [5]:
import numpy as np
from pandas import read_csv    

import time

from datetime import datetime

def day_time(ti):
    return int((int)(ti/(60*60*24)))

t0 = (int)(datetime.strptime(ini_date, '%d/%m/%y %H:%M:%S').timestamp())
print (ini_date,"Init date is", t0, "day:",day_time(t0))

tN = (int)(datetime.strptime(end_date, '%d/%m/%y %H:%M:%S').timestamp())
print (end_date,"End date is", tN, "day:",day_time(tN))


tt = (int)(datetime.strptime(tt_date, '%d/%m/%y %H:%M:%S').timestamp())
print (tt_date,"TT date is", tt_date, "day:",day_time(tt))

nT=(int)(round((tN-t0-1)/time_step))
print(nT)


def absT(tt):
    return (int)(tt*time_step+t0)

def relT(ti):
    return (int)((ti-t0)/time_step)


04/01/23 12:00:00 Init date is 1672830000 day: 19361
06/01/23 13:00:00 End date is 1673006400 day: 19363
05/01/23 14:16:00 TT date is 05/01/23 14:16:00 day: 19362
11760


In [6]:
def repaintXY(x,y):
    return int(x*0.08965-196.41+150), int(-y*0.1078+1077)

def repaintXXYY(x,y):
    return int((x+196.41-150)/0.08965), int((y-1077)/(-0.1078))


print(repaintXY(2280,5135))
print(repaintXY(2860,6730))
print(repaintXXYY(209,351))
#print(repaintXY(5300,6450))
#print(repaintXY(6600,5200))

(157, 523)
(209, 351)
(2848, 6734)


In [7]:
series=read_csv("./sensors-location.tsv", sep="\t",header=None)
print(series.values)
sensor_location={}
for d in series.values:
    for s in d[0].split(","):
        sensor_location[s]=[]
for d in series.values:
    for s in d[0].split(","):
        sensor_location[s].append([float(d[1]),float(d[2]),float(d[3]),float(d[4]),float(d[5])])
    
print(sensor_location)
    

[['cubiertos' 5200 6000 6200 6940 1.0]
 ['fridge' 2600 6700 3960 6000 1.0]
 ['fridge' 1965 6820 4000 5500 0.5]
 ['microwave' 5730 6600 7120 5830 1.0]
 ['microwave' 5780 7060 6920 5650 0.5]
 ['ollas' 6180 8180 6840 7300 1.0]
 ['ollas' 5770 8370 7220 6620 0.5]
 ['rule-cooking' 4840 8710 6300 7350 0.75]
 ['rule-cooking' 5200 8540 6500 7560 1.0]
 ['rule-lunch' 2100 8000 5000 6580 0.75]
 ['rule-lunch' 2570 7690 4780 6800 1.0]]
{'cubiertos': [[5200.0, 6000.0, 6200.0, 6940.0, 1.0]], 'fridge': [[2600.0, 6700.0, 3960.0, 6000.0, 1.0], [1965.0, 6820.0, 4000.0, 5500.0, 0.5]], 'microwave': [[5730.0, 6600.0, 7120.0, 5830.0, 1.0], [5780.0, 7060.0, 6920.0, 5650.0, 0.5]], 'ollas': [[6180.0, 8180.0, 6840.0, 7300.0, 1.0], [5770.0, 8370.0, 7220.0, 6620.0, 0.5]], 'rule-cooking': [[4840.0, 8710.0, 6300.0, 7350.0, 0.75], [5200.0, 8540.0, 6500.0, 7560.0, 1.0]], 'rule-lunch': [[2100.0, 8000.0, 5000.0, 6580.0, 0.75], [2570.0, 7690.0, 4780.0, 6800.0, 1.0]]}


In [8]:
sensor_activation={}
for sensor in binary_sensors:
    sensor_activation[sensor]=[]
    series=read_csv(target_folder+sensor+".tsv", sep="\t",parse_dates=True,header=None)
    sensor_activation[sensor]=  np.array(series.values[:,[1]])  
    print(sensor_activation[sensor].shape)


(11760, 1)


In [9]:
user_location={}
for tag in tags:
    user_location[tag]=[]
    series=read_csv(target_folder+tag+".tsv", sep="\t",parse_dates=True,header=None)
    user_location[tag]=  np.array(series.values[:,[1,2,3,4,5]])  
    print(user_location[tag].shape)
    
    f= open(target_folder+tag+".XY.tsv", "w")
    for d in np.array(series.values[:,[1,2,3,4,5,6]]):
        if( 'None' in d):
            f.write(d[5]+"\t"+d[0]+"\t"+d[1]+"\t"+d[2]+"\t"+d[3]+"\t"+d[4]+"\n")
            continue
        x1,y1=repaintXY(float(d[0]),float(d[1]))
        x2,y2=repaintXY(float(d[2]),float(d[3]))
        f.write(str(d[5])+"\t"+str(x1)+"\t"+str(y1)+"\t"+str(x2)+"\t"+str(y2)+"\t"+str(d[4])+"\n")
        
    f.close()
print(user_location["1"])


(11760, 5)
(11760, 5)
[['None' 'None' 'None' 'None' 'None']
 ['None' 'None' 'None' 'None' 'None']
 ['None' 'None' 'None' 'None' 'None']
 ...
 ['None' 'None' 'None' 'None' 'None']
 ['None' 'None' 'None' 'None' 'None']
 ['None' 'None' 'None' 'None' 'None']]


In [10]:
series=read_csv(target_folder+"1.tsv", sep="\t",parse_dates=True,header=None)
ts=  np.array(series.values[:,[5]])  
print(ts)
print(ts.shape[0])

[['None']
 ['None']
 ['None']
 ...
 ['None']
 ['None']
 ['None']]
11760


In [11]:
from os import listdir
from os.path import isfile, join

pathcamera1="../data-amaltea/fuzzy1C/"
filecamera1= [f for f in listdir(pathcamera1) if isfile(join(pathcamera1, f))]
filecamera1.sort()
pathcamera2="../data-amaltea/fuzzy2C/"
filecamera2= [f for f in listdir(pathcamera2) if isfile(join(pathcamera2, f))]
filecamera2.sort()
           

In [12]:
import cv2
import numpy as np
def findCloserImage(ti, pathcamera, startI=0):
    bef=False
    for idx,f in enumerate(pathcamera):
        if(idx<startI):
            continue
        t2=int(f[0:f.index(".")])
        #print(t2,ti,t2<=ti)
        if(t2<=ti):
            bef=True
        else:
            if bef:
                return f, idx
    return None, 0
      
    



#for file1 in filecamera1:
#    print(file1)
#    im1 = cv2.imread(pathcamera1+file1)
#    cv2.imshow('cam',im1)
#    cv2.waitKey(1)        



In [13]:
val0=0.25
def intervals(s):
    ret=[]
    lastV=0
    lastI=-1
    for t in range(0,len(s)):
        if(lastV>=val0 and s[t]<val0):
            print("\t interval ",t)
            ret.append((lastI,t-1))
        if(lastV<val0 and s[t]>=val0):
            lastI=t
        lastV=s[t]
    print(ret)
    return ret

In [14]:
S=np.zeros((len(binary_sensors), nT))
for i,sensor in enumerate(binary_sensors):
    series=read_csv(target_folder+sensor+".tsv", sep="\t",parse_dates=True,header=None)
    S[i]=  np.array(series.values[:,[1]]).flatten()  
    print(S[i].shape)
print(S.shape)
print(np.where( (S[0]==1)))


(11760,)
(1, 11760)
(array([10996, 10997, 10998, 10999, 11000, 11001, 11002, 11003, 11004,
       11005, 11006, 11007, 11008, 11009, 11010, 11011, 11012, 11013,
       11014, 11015, 11016, 11017, 11018, 11019, 11020, 11021, 11022,
       11023, 11024, 11025, 11026, 11027, 11028, 11029, 11030, 11031,
       11032, 11033, 11034, 11035, 11036, 11037, 11038, 11039, 11040,
       11041, 11042, 11043, 11044, 11045, 11046, 11047, 11048, 11049,
       11050, 11051, 11052, 11053, 11054, 11055, 11056, 11057, 11058,
       11059, 11060, 11061, 11062, 11063, 11064, 11065, 11066, 11067,
       11068, 11069, 11070, 11071, 11072, 11073, 11074, 11075, 11076,
       11077, 11078, 11079, 11080, 11081, 11082, 11083, 11084, 11085,
       11086, 11087, 11088, 11089, 11090, 11091, 11092, 11093, 11094,
       11095, 11096, 11097, 11098, 11099, 11100, 11101, 11102, 11103,
       11104, 11105, 11106, 11107, 11108, 11109, 11110, 11111, 11112,
       11113, 11114, 11115, 11116, 11117, 11118, 11119, 11120, 11121,

In [15]:
for i,sensor in enumerate(binary_sensors):
        print("Evaluating sensor activation of ",sensor, intervals(S[i]))
        for ii in intervals(S[i]):
            print("\t Interval",ii, (ii[0]), (ii[1]))


	 interval  381
	 interval  6404
	 interval  8013
	 interval  11148
	 interval  11534
[(378, 380), (6317, 6403), (7988, 8012), (10977, 11147), (11480, 11533)]
Evaluating sensor activation of  rule-cooking [(378, 380), (6317, 6403), (7988, 8012), (10977, 11147), (11480, 11533)]
	 interval  381
	 interval  6404
	 interval  8013
	 interval  11148
	 interval  11534
[(378, 380), (6317, 6403), (7988, 8012), (10977, 11147), (11480, 11533)]
	 Interval (378, 380) 378 380
	 Interval (6317, 6403) 6317 6403
	 Interval (7988, 8012) 7988 8012
	 Interval (10977, 11147) 10977 11147
	 Interval (11480, 11533) 11480 11533


In [16]:
from tkinter import *
from PIL import ImageTk, Image
from pandas import read_csv
import time
from threading import Thread
import threading

speed=0.1



incT=time_step

wait_step2step=False
wait_event=True


con = threading.Condition()

do_capture=True


lastIm1=0
lastIm2=0


def create_rectangle(x1, y1, x2, y2, **kwargs):
    if(x1>x2):
        aux=x2
        x2=x1
        x1=aux
    if(y1>y2):
        aux=y2
        y2=y1
        y1=aux
        
    if 'alpha' in kwargs:
        alpha = int(kwargs.pop('alpha') * 255)
        fill = kwargs.pop('fill')
        fill = win.winfo_rgb(fill) + (alpha,)
        image = Image.new('RGBA', (x2-x1, y2-y1), fill)
        #images.append()
        canvas.create_image(x1, y1, image=ImageTk.PhotoImage(image), anchor='nw')
    canvas.create_rectangle(x1, y1, x2, y2, **kwargs)
    
def create_text(canvas, text): #center coordinates, radius
    play=">" if incT>0 else "<"
    if(not wait_step2step):
        play=play+play
        
    if(not wait_event):
        play=play+play
        
    
    canvas.create_rectangle(
    5, 5, 200, 20,
    outline="#fb0",
    fill="#fb0")
    canvas.create_text(100,10,fill="darkblue",text="["+play+"]"+text)

def create_big_text(canvas, text, index, back): #center coordinates, radius
    if(back):
        canvas.create_rectangle(
        20, 25, 400, 200,
        outline="#fb0",
        fill="#fb0")
    canvas.create_text(150,50+index*25,fill="darkblue",text=text)    

def create_circle(x, y, r, canvas, color): #center coordinates, radius
    x0 = x - r
    y0 = y - r
    x1 = x + r
    y1 = y + r
    canvas.create_oval(x0, y0, x1, y1, fill=color)

def create_bb(x1, y1, x2, y2, canvas, color, text, alpha=.5): #center coordinates, radius
    create_rectangle(
    x1, y1, x2, y2,    outline=color,
    fill=color, alpha=alpha)
    canvas.create_text((x1+x2)/2,(y1+y2)/2,fill="black",text=text)

def paintUserLocation(tt):
    t=relT(tt)
    for tag in tags:
        if( 'None' in user_location[tag][t]):
            continue

        user_location[tag][t]=user_location[tag][t].astype(np.float)
        x1,y1,x2,y2,w=user_location[tag][t][0],user_location[tag][t][1],user_location[tag][t][2],user_location[tag][t][3],user_location[tag][t][4]
        print("User@paintUserLocation",datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S"), " x1,y1,x2,y2:",x1,y1,x2,y2)
        x1,y1=repaintXY(x1,y1)
        x2,y2=repaintXY(x2,y2)
        print("User#paintUserLocation",datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S"), " x1,y1,x2,y2:",x1,y1,x2,y2)
        create_bb(x1,y1, x2,y2, canvas, colors[tag], tag+"("+str(t)+")",0.5)
    time.sleep(speed)
        
        
def paintSensorActivation(tt):
    t=relT(tt)
    for sensor in binary_sensors:
        if sensor_activation[sensor][t] > 0:
            #limpiamos
            canvas.create_image(400,300, image=img)
            create_text(canvas, datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S"))
            paintUserLocation(tt)
            #paintUserLocation(tt+time_step)
            #paintUserLocation(tt-time_step)
            #ponemos
            
            for sl in sensor_location[sensor]:
                x1,y1,x2,y2,w=sl[0],sl[1],sl[2],sl[3],sl[4]

                print("Sensor@paintSensorActivation", sensor, datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S"), " x1,y1,x2,y2,w:",x1,y1,x2,y2,w)
                x1,y1=repaintXY(x1,y1)
                x2,y2=repaintXY(x2,y2)                
                print("Sensor#paintSensorActivation", sensor, datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S"), " x1,y1,x2,y2,w:",x1,y1,x2,y2,w)
                create_bb(x1,y1, x2,y2, canvas, "#fb0",sensor,w*0.5)

            IoU(tt,sensor)
            #nb = input('Press to continue.. ')
            if(wait_event):
                con.acquire()
                con.wait()
                con.release()
            
def paintAllSensors():
    global do_capture
    for sensor in binary_sensors:
            for sl in sensor_location[sensor]:
                x1,y1,x2,y2,w=sl[0],sl[1],sl[2],sl[3],sl[4]

                print("Sensor@paintSensorActivation", sensor, datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S"), " x1,y1,x2,y2,w:",x1,y1,x2,y2,w)
                x1,y1=repaintXY(x1,y1)
                x2,y2=repaintXY(x2,y2)                
                print("Sensor#paintSensorActivation", sensor, datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S"), " x1,y1,x2,y2,w:",x1,y1,x2,y2,w)
                create_bb(x1,y1, x2,y2, canvas, "#fb0",sensor,w*0.5)

                
    con.acquire()
    con.wait()
    do_capture=False
    con.release()            

def IoU(tt,sensor):
    t=relT(tt)
    iou={}
    for ixtag,tag in enumerate(tags):
        if( 'None' in user_location[tag][t]):
            continue
        
        user_location[tag][t]=user_location[tag][t].astype(np.float)
        x1,y1,x2,y2,w=user_location[tag][t][0],user_location[tag][t][1],user_location[tag][t][2],user_location[tag][t][3],user_location[tag][t][4] 

        x1,y1=repaintXY(x1,y1)
        x2,y2=repaintXY(x2,y2) 

        iou[tag]=0
        for sl in sensor_location[sensor]:
            x1o,y1o,x2o,y2o,wo=sl[0],sl[1],sl[2],sl[3],sl[4]
            x1o,y1o=repaintXY(x1o,y1o)
            x2o,y2o=repaintXY(x2o,y2o)
            print("IoU#",[x1,y1,x2,y2],[x1o,y1o,x2o,y2o])
            if(x1o>x2o):
                aux=x2o
                x2o=x1o
                x1o=aux
            if(y1o>y2o):
                aux=y2o
                y2o=y1o
                y1o=aux
            if(x1>x2):
                aux=x2
                x2=x1
                x1=aux
            if(y1>y2):
                aux=y2
                y2=y1
                y1=aux
            print("IoU#",[x1,y1,x2,y2],[x1o,y1o,x2o,y2o],)
            
            viou=get_iou([x1,y1,x2,y2],[x1o,y1o,x2o,y2o])
            print("IoU:",tag,viou)
            print(w,wo,"F(IoU):",tag,viou*w*wo)
            #iou[tag]=max(iou[tag],viou*w*wo)
            iou[tag]=iou[tag]+viou*w*wo
            
        print("iou[tag]:",iou[tag])
        #if(iou[tag]>0):
        create_big_text(canvas, "Usuario "+str(tag)+" y S:"+str(sensor)+" w:"+str(iou[tag])+" ? ",ixtag, ixtag==0)


def paintSensorArea():
    for sensor in binary_sensors:
            for sl in sensor_location[sensor]:
                x1,y1,x2,y2,w=sl[0],sl[1],sl[2],sl[3],sl[4]

                print("Sensor@paintSensorArea", sensor, datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S"), " x1,y1,x2,y2,w:",x1,y1,x2,y2,w)

                x1,y1=repaintXY(x1,y1)
                x2,y2=repaintXY(x2,y2)                
                create_bb(x1,y1, x2,y2, canvas, "#fb0",sensor,w*0.5)
                print("Sensor#paintSensorArea", sensor, datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S"), " x1,y1,x2,y2,w:",x1,y1,x2,y2,w)

            con.acquire()
            con.wait()
            con.release()            

def paint(tt):
    t=relT(tt)
    create_text(canvas, datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S"))

    paintUserLocation(tt)
    paintSensorActivation(tt)    
        
def print0():
    
    global lastIm1
    global lastIm2
    
    paintAllSensors()
    nextStep()
    

def nextStep():
    global tt
    global lastIm1
    global lastIm2
    
    S=np.zeros((len(binary_sensors), nT))
    for i,sensor in enumerate(binary_sensors):
        series=read_csv(target_folder+sensor+".tsv", sep="\t",parse_dates=True,header=None)
        S[i]=  np.array(series.values[:,[1]]).flatten()  
        print(S[i].shape)
    print(S.shape)
    print(np.where( (S[0]==1)))
    
    
    for ix,sensor in enumerate(binary_sensors):
        print("Evaluating sensor activation of ",sensor, intervals(S[ix]))
        print(ix, S[ix], np.where( (S[ix]==1)))
        inter=intervals(S[ix])
        
        #for jx in range(0,len(inter)):
        jx=0
        while True:
    #    while True:
    
            if(wait_step2step and wait_event):
                tt=tt+incT
            else:
                if(incT>0):
                    jx=jx+1
                else:
                    jx=jx-1
                ii=inter[jx]
                print("\t Interval",ii, (ii[0]), (ii[1])) 
                tt=absT(ii[0])
        
            print("nextStep",tt,tN,t0,incT)
            if(jx>len(inter) or jx<0):
                break
            if(tt>tN or tt<t0):
                break


            t=relT(tt)
            if(t%10==0 or (wait_step2step and wait_event)):
                canvas.delete("all")
                canvas.create_rectangle(0, 0, 800, 800,fill="white")
                canvas.create_image(400, 300, image=img)
            paint(tt)



            file1,lastIm1=findCloserImage(tt,filecamera1,-1)
            file2,lastIm2=findCloserImage(tt,filecamera2,-1)
            print("images:",tt,file1,file2)
            if(file1 is not None and file2 is not None):
                im1 = cv2.imread(pathcamera1+file1)
                cv2.putText(im1, str(tt)+"#"+str(file1), (5,15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)
                im2 = cv2.imread(pathcamera2+file2)
                cv2.putText(im2, str(tt)+"#"+str(file2), (5,15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)

                im1 = cv2.resize(im1, None, fx = 0.5, fy = 0.5, interpolation = cv2.INTER_CUBIC)
                im2 = cv2.resize(im2, None, fx = 0.5, fy = 0.5, interpolation = cv2.INTER_CUBIC)

                hor = np.concatenate((im1, im2), axis=1)
                cv2.imshow('cam',hor)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

            if(wait_step2step):
                con.acquire()
                con.wait()
                con.release()



def callback(event):
    print( "clicked at", event.x, event.y)
    print( " world x,y at", repaintXXYY(event.x, event.y))
    con.acquire()
    con.notify()
    con.release()

    
def motion(event):
    global do_capture
    if do_capture:
        x, y = event.x, event.y
        print('{}, {}'.format(x, y))    
        print(repaintXXYY(x, y))
        create_text(canvas, str(repaintXXYY(x, y)))
        
def key(event):
    print ("pressed", repr(event.char), str(repr(event.char)))   
    global incT
    global wait_step2step
    global wait_event
    
    if(str(repr(event.char)) == "'a'"):
        print('a!')
        incT=-time_step
        wait_step2step=False
        wait_event=False
    if(repr(event.char) == "'s'"):
        print('s!')
        incT=-time_step
        wait_step2step=False
        wait_event=True
    if(repr(event.char) == "'d'"):
        print('d!')
        incT=-time_step
        wait_step2step=True
        wait_event=True   
    if(repr(event.char) == "'l'"):
        print('l!')
        incT=time_step
        wait_step2step=False
        wait_event=False
    if(repr(event.char) == "'k'"):
        print('k!')
        incT=time_step
        wait_step2step=False
        wait_event=True
    if(repr(event.char) == "'j'"):
        print('j!')
        incT=time_step
        wait_step2step=True
        wait_event=True    
    create_text(canvas, datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S"))
    con.acquire()
    con.notify()
    con.release()
        
#Create an instance of tkinter window
win = Tk()

# Define the geometry of the window
#win.geometry("1010x612")

#frame = Frame(win, width=600, height=400)
#frame.pack()
#frame.place(anchor='center', relx=0.5, rely=0.5)

# Create an object of tkinter ImageTk
img = ImageTk.PhotoImage(Image.open("plano5.png"))

# Create a Label Widget to display the text or Image
#label = Label(frame, image = img)

#label.pack()
#label.bind('<Button 1>', motion)

canvas = Canvas(win, height=600, width=700)
canvas.create_image(400, 300, image=img)
canvas.bind("<Button-1>", callback)
win.bind("<Key>", key)
win.bind("<Motion>", motion)

#print0() 

canvas.pack()
print("pre loop")



thread = Thread(target = print0, args = ())
thread.setDaemon(True)
thread.start()



    
win.mainloop()


    




pre loop
Sensor@paintSensorActivation rule-cooking 2023-01-05 14:16:00  x1,y1,x2,y2,w: 4840.0 8710.0 6300.0 7350.0 0.75
Sensor#paintSensorActivation rule-cooking 2023-01-05 14:16:00  x1,y1,x2,y2,w: 387 138 518 284 0.75
Sensor@paintSensorActivation rule-cooking 2023-01-05 14:16:00  x1,y1,x2,y2,w: 5200.0 8540.0 6500.0 7560.0 1.0
Sensor#paintSensorActivation rule-cooking 2023-01-05 14:16:00  x1,y1,x2,y2,w: 419 156 536 262 1.0
646, 300
(7723, 7207)
602, 302
(7232, 7189)
574, 303
(6920, 7179)
561, 306
(6775, 7152)
551, 307
(6663, 7142)
546, 308
(6608, 7133)
540, 310
(6541, 7115)
534, 310
(6474, 7115)
529, 311
(6418, 7105)
523, 311
(6351, 7105)
518, 312
(6295, 7096)
511, 312
(6217, 7096)
504, 313
(6139, 7087)
499, 313
(6083, 7087)
494, 313
(6027, 7087)
491, 313
(5994, 7087)
489, 313
(5972, 7087)
487, 313
(5949, 7087)
484, 313
(5916, 7087)
480, 313
(5871, 7087)
475, 313
(5816, 7087)
469, 314
(5749, 7077)
465, 315
(5704, 7068)
464, 315
(5693, 7068)
464, 316
(5693, 7059)
463, 316
(5682, 7059)
c

C:\Users\javie\AppData\Local\Temp\ipykernel_19756\4176740027.py:88: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  user_location[tag][t]=user_location[tag][t].astype(np.float)


Sensor@paintSensorActivation rule-cooking 2023-01-05 14:19:15  x1,y1,x2,y2,w: 4840.0 8710.0 6300.0 7350.0 0.75
Sensor#paintSensorActivation rule-cooking 2023-01-05 14:19:15  x1,y1,x2,y2,w: 387 138 518 284 0.75
Sensor@paintSensorActivation rule-cooking 2023-01-05 14:19:15  x1,y1,x2,y2,w: 5200.0 8540.0 6500.0 7560.0 1.0
Sensor#paintSensorActivation rule-cooking 2023-01-05 14:19:15  x1,y1,x2,y2,w: 419 156 536 262 1.0
IoU# [533, 420, 564, 391] [387, 138, 518, 284]
IoU# [533, 391, 564, 420] [387, 138, 518, 284]
 iou: [533, 391, 518, 284]
IoU: 1 0.0
1.0 0.75 F(IoU): 1 0.0
IoU# [533, 391, 564, 420] [419, 156, 536, 262]
IoU# [533, 391, 564, 420] [419, 156, 536, 262]
 iou: [533, 391, 536, 262]
IoU: 1 0.0
1.0 1.0 F(IoU): 1 0.0
iou[tag]: 0.0
IoU# [342, 485, 754, 217] [387, 138, 518, 284]
IoU# [342, 217, 754, 485] [387, 138, 518, 284]
 iou: [387, 217, 518, 284]
 intersection_area/bb1_area: 8777 110416
IoU: 2 0.4589041095890411
1.0 0.75 F(IoU): 2 0.3441780821917808
IoU# [342, 217, 754, 485] [419, 1

C:\Users\javie\AppData\Local\Temp\ipykernel_19756\4176740027.py:151: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  user_location[tag][t]=user_location[tag][t].astype(np.float)


clicked at 483 314
 world x,y at (5905, 7077)
images: 1672924755 1672924758.5677857.png 1672924757.0821652.png
	 Interval (7988, 8012) 7988 8012
nextStep 1672949820 1673006400 1672830000 15
User@paintUserLocation 2023-01-05 21:17:00  x1,y1,x2,y2: 5974.24370196864 7950.351927411695 6903.8605895527035 8866.093417393982
User#paintUserLocation 2023-01-05 21:17:00  x1,y1,x2,y2: 489 219 572 121
User@paintUserLocation 2023-01-05 21:17:00  x1,y1,x2,y2: 2276.3862799593226 4782.694641103051 7636.775339173172 6848.228515297656
User#paintUserLocation 2023-01-05 21:17:00  x1,y1,x2,y2: 157 561 638 338
User@paintUserLocation 2023-01-05 21:17:00  x1,y1,x2,y2: 5974.24370196864 7950.351927411695 6903.8605895527035 8866.093417393982
User#paintUserLocation 2023-01-05 21:17:00  x1,y1,x2,y2: 489 219 572 121
User@paintUserLocation 2023-01-05 21:17:00  x1,y1,x2,y2: 2276.3862799593226 4782.694641103051 7636.775339173172 6848.228515297656
User#paintUserLocation 2023-01-05 21:17:00  x1,y1,x2,y2: 157 561 638 338
